This is a follow up to `StateAndChannelDemo.ipynb` that delves in to the different forms of CPTP maps included in the `QuantumChannel` class.

In [ ]:
import mezze.channel as ch
import numpy as np

# Representations Supported

`QuantumChannel` support a number of different representations of CPTP maps on which are given in the first argument (`rep`) to the constructor, and the type is specififed by the second argumeny (`in_type`).  These representations currently supported are (in terms of qudit dimension $N$):

* Liouvillian: `rep` = $N^2\times N^2$ matrix, `in_type` = `liou`
* Choi matrix: `rep` = $N^2\times N^2$ matrix, `in_type` = `choi`
* $\chi$ matrix: `rep` = $N^2\times N^2$ matrix, `in_type` = `chi` 
  * `basis` = `list` of $N\times N$ matrices (optional) -- Defaults to standard Pauli
* Kraus operators: `rep` = `list` of $N\times N$ matrices, `in_type`=`kraus`
* Unitary operation: `rep` = $N\times N$ matrix, `in_type`=`unitary`
* Pauli transfer matrix: `rep`= $N^2\times N^2$ matrix, `in_type`=`ptm`

Additionally, two specific representations of the Stinespring form are included that have a relationship to random CPTP maps.  These are unlikely to be outside of development of random CPTP map code:

* Stiefel form: `\rep` = $KN\times N$ matrix, `in_type`=`stiefel`
* Stiefel-2 form: `\rep` = $KN\times N$ matrix, `in_type`=`stiefel2`

Duck-typing is permitted, so that essentially any 2-dimensional array-like structure will work in place as a matrix representation, e.g., `numpy.array`, `numpy.matrix`, and `list` of `list`.  Currently, there is no input checking (other than type and size), so specific constraints that ensure CPTP-ness are not.     That said, the method `QuantumChannel.is_valid()` can be used to check the validity of the input.  Note that this allows for some "abuse" of the methods of `QuantumChannel` to perform matrix transforms.

# Changing Representations

`QuantumChannel` objects are created using one of the above CPTP map representations, and it will automatically create interneal (`numpy.matrix`-based) representations of a given form when requested, or when they are needed to perform another operation, such as composition.  The `QuantumChannel` methods to retrieve this concrete representations are

* `liouvillian()`: returns $N^2\times N^2$ `numpy.matrix`
* `choi()`: returns $N^2\times N^2$ `numpy.matrix`
* `chi()`: returns $N^2\times N^2$ `numpy.matrix`
* `kraus()`: returns `list` of $N\times N$ `numpy.matrix`
* `ptm()`: returns $N^2\times N^2$ `numpy.matrix`
* `stiefel()`: returns $N^3\times N$ `numpy.matrix`
* `stiefel2()`: returns $N^3\times N$ `numpy.matrix`

The exception to this is the `unitary` `in_type`, which cannot be recovered directly. Instead `QuantumChannel.kraus()[0]` will return the unitary matrix representation of a unitary operation.

# Example 1: Unitary Operations

The easiest way to define unitary operations is through the `unitary` `in_type`

In [ ]:
I = ch.QuantumChannel(np.eye(2),'unitary')
X = ch.QuantumChannel([[0,1],[1,0]],'unitary')
Y = ch.QuantumChannel(np.matrix([[0,-1j],[1j,0]]),'unitary')
Z = ch.QuantumChannel(np.array([[1,0],[0,-1]]),'unitary')
H = ch.QuantumChannel(1/np.sqrt(2)*np.array([[1,1],[1,-1]]),'unitary')
CNOT = ch.QuantumChannel([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]],'unitary')

# Example 2: Pauli Channels

By default $\chi$ matrices and Pauli transfer matrices are expressions in the Pauli basis of the Choi and Liouvillian forms, respectively, thus Pauli channels are expressed as diagonal matrices in these forms.

In [ ]:
Z_depol = ch.QuantumChannel(np.diag([.99,0,0,.01]),'chi')
Z_depol2 = ch.QuantumChannel(np.diag([1,.98,.98,1]),'ptm')

print('Z_depol in chi-matrix form:')
print(Z_depol.chi())

print('\n Z_depol2 in chi-matrix form (same as Z_depol)')
print(Z_depol2.chi())

uniform_depolarizing = ch.QuantumChannel(np.diag([.97,.01,.01,.01]),'chi')
non_uniform_depolarizing = ch.QuantumChannel(np.diag([1,.7,.8,.9]),'ptm')

print('\nUniform depolarizing chi->ptm')

print(uniform_depolarizing.ptm())

print('\n Non-uniform depolarizing ptm->chi')
print(np.round(non_uniform_depolarizing.chi(),2))

# Example 3: Amplitude Damping

Amplitude damping is expressed in Kraus form (in terms of decay $\gamma$) via $$A_1=\begin{bmatrix}1&0\\0&\sqrt{1-\gamma}\end{bmatrix}$$ $$A_2=\begin{bmatrix}0&\sqrt{\gamma}\\0&0\end{bmatrix}$$

In Pauli transfer matrix form, this is $$\begin{bmatrix}1&0&0&0\\0&\sqrt{1-\gamma}&0&0\\0&0&\sqrt{1-\gamma}&0\\\gamma&0&0&1-\gamma\end{bmatrix}$$

In [ ]:
gamma = .1**2
A1 = [[1,0],[0,np.sqrt(1-gamma)]]
A2 = [[0,np.sqrt(gamma)],[0,0]]
amp_damp_kraus = ch.QuantumChannel([A1,A2],'kraus')

print('sqrt(1-\gamma): {0}'.format(np.sqrt(1-gamma)))

print('\nKraus definition agrees with PTM version:')
amp_damp_kraus.ptm()

# Other Operations

* `is_valid()` checks CP and TP conditions based on `in_type`
* `rank()` returns the Kraus rank of the CPTP map
* `is_unital()` returns unitality of CPTP map
* `is_extremal()` returns `True` if CPTP map is an extremal element of the convex set of CPTP maps
* `stiefel_tangent()` returns the tangent space element of Stiefel manifold to the CPTP map in Stiefel form.